# Retrieval-Augmented Generation (RAG)

**2 Implementierung eines KI-Systems mit RAG  und Agentenkomponenten**

**2.1 RAG-System**

Wie bereits im ersten Teil des Assignments beschrieben, kommen RAG-Systeme (Retrieval-Augmented Generation) immer dann zum Einsatz, wenn externe Wissensquellen in ein Sprachmodell eingebunden werden sollen. Die Motivation liegt darin, dass ein LLM allein nur auf seinem Trainingswissen basiert, während sich durch RAG zusätzlich aktuelle oder spezialisierte Informationen berücksichtigen lassen. Das Vorgehen umfasst im Wesentlichen die Indexierung externer Daten, das semantische Retrieval relevanter Inhalte und die Generierung einer Antwort durch das LLM. Dieses Notebook zeigt exemplarisch, wie ein solches System aufgebaut wird und wie die einzelnen Bausteine zusammenwirken.



---



**Auswahl der externen Quelle**

Da eines meiner Hobbys das Anschauen von Sport ist, insbesondere **Tennis**, und wir uns derzeit in der Hochsaison befinden, in der gerade Wimbledon zu Ende gegangen ist und nun Cincinnati und die US Open direkt hintereinander folgen, habe ich beschlossen, diese Aufgabe auf diesen Bereich zu konzentrieren. Darüber hinaus hat sich nach der Ära der „Big Three“ eine neue Rivalität im Herrentennis entwickelt, die noch mehr Aufmerksamkeit und Begeisterung für diesen Sport mit sich bringt.  

Auf der Suche nach geeigneten Videos oder Dokumenten mit interessanten und relevanten Informationen für diese RAG-Aufgabe bin ich auf ein sehr gutes YouTube-Video gestoßen:  
**„Every TENNIS ERA Explained In 10 Minutes”** ([Link](https://www.youtube.com/watch?v=dxg1-JXKG5A)).  

Ich habe dieses Video transkribiert und die Transkription in mein GitHub-Repository hochgeladen ([Raw-Link](https://raw.githubusercontent.com/semauenal/assignment-rag-agentensysteme/refs/heads/main/Transcript_Every_Tennis_Era_Explained.txt)). Die Transkription umfasst **1.508 Wörter** und dient als externe Wissensquelle für das in diesem Notizbuch implementierte RAG-System.  



---



**Erklärungen zum Code**

Bevor endgültig der Code zum RAG-System gestartet wird, soll nochmal kurz erklärt werden, auf was Wert gelegt wird beim Implementieren des RAGs.

- Kommentare: Es wird versucht, verständliche, aber nicht zu viele Kommentare zu setzen.

- Erklären von Abstrahierung und Frameworks bzw. Funktionen: Vorhandene Abstraktionen oder Framework-Aufrufe werden kurz erläutert, damit klar ist, was im Hintergrund passiert.

- Immer wieder Print-Ausgaben: Zur Kontrolle werden Zwischenergebnisse ausgegeben, um den Ablauf und die Daten zu prüfen.

- Tokennutzung / Limits: Durch Parameter wie max_tokens und die Auswahl weniger Chunks wird der Tokenverbrauch begrenzt und kontrollierbar gehalten.



---



**Verbindung zu OpenAI**

In diesem Abschnitt wird die Verbindung zur OpenAI-API hergestellt. Dafür wird der API-Schlüssel aus den Secrets geladen.

Nutzung der OpenAI-Python-Bibliothek:
- Zugriff auf OpenAI-API vereinfacht
- Anstatt selbst HTTP-Anfragen an die Endpunkte (z. B. /v1/chat/completions, /v1/embeddings) zu schreiben, bietet die Bibliothek eine fertige Schnittstelle in Python
- Klasse OpenAI erstellt einen Client, der mit dem API-Key authentifiziert wird

  --> „Brücke“ zwischen dem Notebook und der OpenAI-Cloud dar und übernimmt die ganze low-level Arbeit der API-Kommunikation.



In [49]:
# API-Key aus Colab-Secrets laden
from google.colab import userdata
from openai import OpenAI
OPENAI_API_KEY = userdata.get('apikey_sm')
client = OpenAI(api_key=OPENAI_API_KEY)

**Modellkonfiguration**

Damit später bei der Generierung des Outputs nicht ständig erneut die Konfigurationen angegeben werden müssen, werden sie hier einmal zentral definiert.


In [50]:
# Konfiguration durch Variabeln
model_chat    = "gpt-4o-mini"
temperature   = 0               # steuert Kreativität
token_limit   = 4096            # internes Budget (Kontrolle, nicht das echte Kontextfenster)

top_k         = 4               # Anzahl der Chunks, die beim Retrieval zurückgegeben werden


In [51]:
# Transkript laden

# Importieren von requests: Python-Bibliothek für HTTP-Anfragen
import requests
url = "https://raw.githubusercontent.com/semauenal/assignment-rag-agentensysteme/refs/heads/main/Transcript_Every_Tennis_Era_Explained.txt"
transcript = requests.get(url).text
print(transcript[:500])  # Vorschau

﻿Back in the day, way before 1968, tennis wasn’t even a competition. It was more about tradition and having fun. The sport lived in black and white. And the players, well, it looked like they were heading to a party and not a professional game. To be fair to them, it wasn’t really a professional sport at the time, but those outfits were still outrageous. I’m talking about the pre-Open Era. This was one of the weirdest eras in the history of this sport. Why? Because this was the time when amateur


**Chunking**

Beim Aufbau eines RAG-Systems ist **Chunking** wichtig, weil große Dokumente nicht auf einmal in den Prompt passen.  
Die Texte werden in Abschnitte zerlegt, damit sie als Embeddings berechnet und später gezielt abgerufen werden können.  
Dabei gilt:  
- Chunks dürfen nicht zu kurz sein, sonst geht inhaltlicher Kontext verloren.  
- Eine **Überlappung** ist sinnvoll, damit am Rand keine relevanten Informationen abgeschnitten werden.  

Hier wurde bewusst eine eigene kleine Funktion implementiert, um den Ablauf nachvollziehbar zu machen.  
Es gibt zwar viele Frameworks (z. B. in LangChain oder LlamaIndex), die das Chunking mit einer einzigen Zeile erledigen,  
aber durch die eigene Funktion wird deutlich, wie der Prozess intern funktioniert.


Die Funktion `chunk_text` arbeitet auf Basis einer Schleife:  
- Der Text wird zuerst in Wörter aufgesplittet.  
- Mit einem Fenster von `chunk_size` Wörtern wird ein Abschnitt gebildet.  
- Jeder Abschnitt wird als String gespeichert und in die Liste `chunks` geschrieben.  
- Der Startindex verschiebt sich dann jeweils um `chunk_size - overlap`, sodass sich die Abschnitte leicht überlappen.  
- Am Ende liefert die Funktion eine Liste von Textabschnitten zurück.  



In [52]:
# Chunking
# Text in Abschnitte von ca. 500 Wörtern mit Überlappung 30 Wörter
def chunk_text(text, chunk_size=300, overlap=30):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(transcript)
print("Anzahl Chunks:", len(chunks))
# Vorschau der ersten 300 Zeichen des ersten Chunks
print("Beispiel Chunk:\n", chunks[0][:300])

Anzahl Chunks: 6
Beispiel Chunk:
 ﻿Back in the day, way before 1968, tennis wasn’t even a competition. It was more about tradition and having fun. The sport lived in black and white. And the players, well, it looked like they were heading to a party and not a professional game. To be fair to them, it wasn’t really a professional spo


**Embedding**

Nach dem Chunking werden die Textabschnitte in sogenannte Embeddings umgewandelt.  
Ein Embedding ist ein dichter Vektor, der die semantische Bedeutung eines Textes in einer hochdimensionalen Zahlen­darstellung abbildet. So können Inhalte mathematisch verglichen werden (z. B. über Cosine-Similarity), was für das semantische Retrieval die Grundlage bildet.  

Um die Chunks in Embeddings umzuwandeln wird der OpenAI-Client genutzt,  weil er einiges schon übernimmt, was man mit requests selbst machen müsste, wie zum Beispiel Header setzen, JSON bauen und Antworten parsen. Dadurch spart man Code und Fehlerquellen, abstrahiert aber auch nicht zu viel. LangChain wäre zwar noch kürzer, nimmt einem aber viele Details ab und versteckt den Ablauf stärker.


Gewähltes Modell  

Für die Embedding-Erstellung wird das Modell **`text-embedding-3-small`** von OpenAI genutzt.  
- Vorteil: kostengünstig und gleichzeitig qualitativ gut genug für ein mittellanges Transkript.  
- Output: pro Textabschnitt ein Vektor mit 1536 Dimensionen.  
  - Jede Dimension ist ein Zahlenwert, der eine bestimmte latente Eigenschaft des Textes repräsentiert.  
  - Man kann es sich wie einen „Punkt im 1536-dimensionalen Raum“ vorstellen, der die Bedeutung des Textes mathematisch beschreibt.  

Abstraktionen im Code  

- **`client.embeddings.create(...)`**:  
  API-Aufruf an den OpenAI-Endpunkt `/v1/embeddings`.  
  Abstrahiert werden: Authentifizierung, Aufbau der Anfrage, Versand und Empfang der JSON-Antwort.  
- **`np.array(vector)`**:  
  Umwandlung der zurückgegebenen Python-Liste in ein NumPy-Array → effizientere Verarbeitung.  
- **`np.vstack(embeddings)`**:  
Nimmt die einzelnen Embedding-Vektoren (jeder ist eine Liste mit 1536 Zahlen) und legt sie untereinander in eine Tabelle.
Ergebnis: eine 2D-Matrix mit der Form (Anzahl Chunks, 1536).
  - Jede Zeile = ein Chunk-Embedding
  - Jede Spalte = eine bestimmte Dimension des Vektorraums


Zusammenfassung  

In diesem Schritt wird also für jeden Chunk ein Embedding erzeugt und zu einer gemeinsamen Matrix zusammengefügt.  
Dies ist der zentrale Zwischenschritt, um später Anfragen mit denselben Methoden in denselben Vektorraum einzubetten und semantisch passende Chunks zurückzufinden.


In [53]:
import numpy as np
# Embeddings berechnen
embeddings_transcript = []
for chunk in chunks:
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=chunk
    )
    vector = response.data[0].embedding
    embeddings_transcript.append(np.array(vector))

embeddings_transcript = np.vstack(embeddings_transcript)

# shape von NumPy-Arrays zeigt Form des Arrays an (Dimensionen)
print(f"Anzahl Embeddings: {embeddings_transcript.shape[0]}")
print(f"Länge pro Embedding-Vektor: {embeddings_transcript.shape[1]} Dimensionen")
# Als Test anzeigen der ersten 10 Dimensionen des ersten Chunks
print(embeddings_transcript[0][:10])

Anzahl Embeddings: 6
Länge pro Embedding-Vektor: 1536 Dimensionen
[-0.00377981  0.02224585 -0.00795002  0.01334467  0.04593974 -0.00183667
  0.00059181 -0.02226005 -0.02113853  0.01954852]


**Retrieval**

Nach Umwandlung der Chunks in Embeddings, folgt Retrieval-Schritt. Ziel: Aus allen gespeicherten Embeddings die relevantesten Chunks zu einer Anfrage finden.

Für diesen Code wurde bewusst ein experimenteller Ansatz gewählt. Statt sofort auf integrierte Lösungen zurückzugreifen, werden hier zwei Hilfsfunktionen selbst definiert. Dadurch wird die Logik nachvollziehbarer.


---


**Cosine-Similarity**

Die Cosine-Similarity kommt aus der linearen Algebra und Geometrie.
Sie misst den Winkel zwischen zwei Vektoren.
Je kleiner der Winkel, desto ähnlicher die Vektoren.
- Formal:  

$$
\cos(\theta) = \frac{a \cdot b}{||a|| \cdot ||b||}
$$  

mit $a \cdot b$ = Skalarprodukt und $||a||$ = Länge (Norm) des Vektors.  

*Wertebereich*
- **1** → Vektoren zeigen exakt in dieselbe Richtung → Inhalte sehr ähnlich  
- **0** → Vektoren stehen senkrecht (orthogonal) → keine Ähnlichkeit  
- **-1** → Vektoren zeigen in entgegengesetzte Richtungen → theoretisch maximale Gegensätzlichkeit (bei Embeddings fast nie relevant)  

*Umsetzung im Code*
- `a` und `b` = zwei Vektoren (z. B. Query-Embedding und ein Chunk-Embedding)  
- `np.dot(a, b)` = Skalarprodukt (Summe der komponentenweisen Multiplikationen)  
- `np.linalg.norm(a)` = Länge des Vektors *a*, berechnet als Wurzel der Summe der Quadrate aller Dimensionen  

- Eine kleine Konstante `1e-12` wird hinzugefügt, um Division durch Null zu vermeiden.  

**Retrieval-Funktion**
retrieve_top_k:
- Query wird ebenfalls in ein Embedding umgewandelt
- Query-Embedding wird mit allen Chunk-Embeddings verglichen
- Die besten Treffer (Top-k) werden sortiert und zurückgegeben.


In [54]:
# Cosine Similarity zwischen zwei Vektoren
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / ((np.linalg.norm(a) * np.linalg.norm(b)) + 1e-12))

# Retrieval-Funktion: Top-k ähnlichste Chunks
def retrieve_top_k(query: str, embeddings: np.ndarray, chunks: list[str], k: int):
    # Query → Embedding
    embedding_response = client.embeddings.create(model="text-embedding-3-small", input=query)
    q_vec = np.array(embedding_response.data[0].embedding, dtype=np.float32)

    # Ähnlichkeiten berechnen
    sims = [cosine_similarity(q_vec, emb) for emb in embeddings]

    # Sortieren & Top-k wählen
    top_ids = np.argsort(sims)[::-1][:k]
    return [(i, sims[i], chunks[i]) for i in top_ids]


# Testaufruf (zur Kontrolle)
query_test = "Who are the Big Three in tennis?"
results = retrieve_top_k(query_test, embeddings_transcript, chunks, top_k)

for idx, score, txt in results:
    print(f"Chunk {idx} | Score = {score:.3f}\n'{txt[:200]}...'\n")


Chunk 2 | Score = 0.624
'set the tone for what was coming next, as it was the first time the best players were put up against each other for the world to watch. But what came next was something that nobody expected. After the...'

Chunk 3 | Score = 0.576
'than you’ll ever find anywhere else. But let’s not forget about one other guy who still left his mark in this era despite playing with such greats. Had he not played in the same era as the Big Three, ...'

Chunk 4 | Score = 0.515
'with a clean game and a cheeky celebration at the end. Alexander Zverev, with his massive serve and lethal forehand, also showed promise. He won an Olympic gold but is still hunting that all-important...'

Chunk 1 | Score = 0.498
'in 1968. The pros and amateurs were allowed to play together in the same tournaments. And let me say this, things got really spicy really quickly. Money started flowing into the sport. Television came...'



**Generierung des Outputs**

Nun geht es um den letzten Teil der RAG-Pipeline. Es wurde bewusst die Variante über den **OpenAI-Client** gewählt, anstatt Frameworks wie LangChain zu nutzen.  Ziel war es, später mit nur einer Zeile (`ask(...)`) Antworten generieren zu können.  Dabei lag der Fokus besonders auf dem Einbau von Kontrollmechanismen, um sicherzustellen,  dass das Tokenbudget nicht überschritten wird.  


Im Folgenden wird der Code etwas detaillierter erklärt:

- **Parameter**:  
  - `question`: Nutzerfrage  
  - `embeddings`, `chunks_in`: vorbereitete Datenbasis aus Embeddings + Textabschnitten  
  - `k`: wie viele Top-Chunks beim Retrieval genutzt werden  
  - `max_ctx_chars`: begrenzt die Länge des eingebauten Kontexts  
  - `max_answer_tokens`: maximale Länge der Modell-Antwort  

- **Ablauf**:  
  1. Die Nutzerfrage wird eingebettet und mit allen Chunks verglichen → Top-k relevante Chunks.  
  2. Der Kontext wird aus diesen Chunks zusammengesetzt, aber auf `max_ctx_chars` Zeichen begrenzt.  
  3. Prompt wird erstellt:  
     - *System-Message*: definiert Regeln („antworte nur aus dem Kontext“).  
     - *User-Message*: enthält Kontext + Nutzerfrage.  
  4. Mit `client.chat.completions.create` wird das LLM aufgerufen.  
  5. Die erste Modellantwort wird extrahiert und als String zurückgegeben.  

Damit ist die komplette RAG-Pipeline abgeschlossen: **eine Nutzerfrage genügt, und das System führt Retrieval, Kontextbau und Antwortgenerierung automatisch aus.*


In [55]:
# Pipeline-Funktion: Retrieval → LLM
def ask(question: str,
        embeddings: np.ndarray = embeddings_transcript,
        chunks_in: list[str] = chunks,
        k: int = top_k,
        max_context_chars: int = 3000,
        max_answer_tokens: int = 300):

    # Top-k Chunks holen
    retrieved = retrieve_top_k(question, embeddings, chunks_in, k)

    # Kontext zusammenbauen (begrenzt auf max_context_chars)
    context_parts, used = [], 0
    for idx, score, txt in retrieved:
        if used + len(txt) > max_context_chars:
            txt = txt[: max_context_chars - used]
        context_parts.append(txt.strip())
        used += len(txt)
        if used >= max_context_chars:
            break
    context = "\n\n---\n\n".join(context_parts)

    # Prompt definieren
    system_msg = (
        "You are a concise assistant. Answer ONLY using the provided context. "
        "If the answer is not in the context, say you don't know."
    )
    user_msg = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer in 3–6 sentences."

    # OpenAI-Call
    completion = client.chat.completions.create(
        model=model_chat,
        temperature=temperature,
        max_tokens=min(max_answer_tokens, token_limit),
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
    )

    return completion.choices[0].message.content

In [56]:
# Beispielfragen


#answer = ask("Fasse die Entwicklung des Tennissports von der Pre-Open Era bis heute in 3–4 Sätzen zusammen.")
#answer = ask("Welche Spielerinnen prägten das moderne Frauentennis und wie unterschieden sie sich?")
#answer = ask("Welche jungen Spieler werden als die Zukunft des Tennissports genannt und warum?")
answer = ask("Welche Spieler gehören zu den Big Three und wodurch zeichnen sie sich jeweils aus?")
print(answer)

Die Spieler, die zu den Big Three gehören, sind Roger Federer, Rafael Nadal und Novak Djokovic. Roger Federer zeichnet sich durch seinen butterweichen Spielstil und die schönste einhändige Rückhand aus; er gewann 20 Grand Slams und dominierte besonders auf Rasen. Rafael Nadal, der König des Sandplatzes, ist bekannt für seine unermüdliche Energie und seinen Kampfgeist, was ihm 14 French Open Titel einbrachte. Novak Djokovic ist für seine außergewöhnliche mentale Stärke bekannt und hat zahlreiche Rekorde gebrochen, darunter die meisten Wochen als Nummer eins der Welt. Gemeinsam haben sie über 60 Grand Slam Titel gewonnen und zahlreiche unvergessliche Duelle ausgetragen.
